# 2. 鏈的內部運作 (Chains Under the Hood) - Gemini 版本

本範例深入理解 LCEL 鏈的內部實作，展示如何使用 RunnableSequence 手動組合鏈的各個步驟。

## 學習重點
- 了解鏈的底層運作機制
- 學習 RunnableSequence 的使用方法
- 理解 RunnableLambda 的作用
- 對比手動組合與 LCEL 語法的差異


In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableSequence
from langchain_google_genai import ChatGoogleGenerativeAI

# 載入環境變數
load_dotenv()

# 建立 Gemini 模型
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


In [ ]:
# 定義提示模板
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個喜劇演員，專門講關於 {topic} 的笑話。"),
        ("human", "請告訴我 {joke_count} 個笑話。"),
    ]
)


In [ ]:
# 手動建立各個可執行步驟 (Runnable)
# 這些步驟對應 LCEL 鏈中的各個元件

# 步驟 1: 格式化提示
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))

# 步驟 2: 呼叫模型
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))

# 步驟 3: 解析輸出
parse_output = RunnableLambda(lambda x: x.content)

print("各個步驟已定義完成：")
print("1. format_prompt: 格式化提示模板")
print("2. invoke_model: 呼叫 LLM 模型")
print("3. parse_output: 解析模型輸出")


In [ ]:
# 使用 RunnableSequence 手動組合鏈
# 這相當於 LCEL 的 prompt_template | model | StrOutputParser()
chain = RunnableSequence(
    first=format_prompt, 
    middle=[invoke_model], 
    last=parse_output
)

print("手動組合的鏈已建立完成！")
print("鏈的結構：format_prompt → invoke_model → parse_output")


In [ ]:
# 執行手動組合的鏈
response = chain.invoke({"topic": "律師", "joke_count": 3})

# 輸出結果
print("=" * 50)
print("笑話結果：")
print("=" * 50)
print(response)


## 💡 重點說明

### 手動組合 vs LCEL 語法

**手動組合 (RunnableSequence)**:
```python
chain = RunnableSequence(
    first=format_prompt, 
    middle=[invoke_model], 
    last=parse_output
)
```

**LCEL 語法**:
```python
chain = prompt_template | model | StrOutputParser()
```

### 優缺點比較

**手動組合的優點**:
- 更清楚地看到每個步驟的實作細節
- 可以對每個步驟進行更精細的控制
- 適合學習和理解鏈的內部運作

**LCEL 語法的優點**:
- 程式碼更簡潔易讀
- 自動具備串流、批次處理等功能
- 是 LangChain 推薦的現代化做法
